In [3]:
import numpy as np
from keras.layers import Activation, Dense
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.utils.vis_utils import plot_model

In [4]:
ALICE_DIR = '/Users/dsp/nltk_data/corpora/gutenberg/carroll-alice.txt'

In [6]:
with open(ALICE_DIR) as f:
    lines = []
    for line in f:
        line = line.strip().lower()
        if len(line) == 0:
            continue
        lines.append(line)
    text = ' '.join(lines)

In [7]:
text[:50]

"[alice's adventures in wonderland by lewis carroll"

In [9]:
chars = set([c for c in text])
n_chars = len(chars)

char2index = {c: i for i, c in enumerate(chars)} 
# 'a': 40, 'b': 18, 'c': 10...

index2char = {i: c for i, c in enumerate(chars)} 
# 0: ')', 1: 'u', 2: 'r'...

In [11]:
SEQ_LEN = 10
STEP = 1

In [12]:
input_chars = []
label_chars = []
for i in range(0, len(text) - SEQ_LEN, STEP) :
    input_chars.append(text[i:i + SEQ_LEN])
    label_chars.append(text[i + SEQ_LEN])

In [14]:
print(f'{input_chars[0]} -> {label_chars[0]}')
print(f'{input_chars[1]} -> {label_chars[1]}')
print(f'{input_chars[2]} -> {label_chars[2]}')

[alice's a -> d
alice's ad -> v
lice's adv -> e
